# Retrieve fits information

In [5]:
from astropy.io import fits
import os

fits_dir = '/workspace/raid/OM_DeepLearning/XMM_OM_dataset/scaled_raw_1024-1000/'
fits_files = [f for f in os.listdir(fits_dir) if os.path.isfile(os.path.join(fits_dir, f)) and f.endswith('.fits')]

fits_info = {}
for filename in fits_files:
    with fits.open(fits_dir+filename) as hdul:
        header = hdul[0].header
        exposure = header.get('EXPOSURE', None)
        ra = header.get('RA_PNT', None)
        ra_scx = header.get('RA_SCX', None)
        dec = header.get('DEC_PNT', None)
        dec_scx = header.get('DEC_SCX', None)
        pa = header.get('PA', None)

        if ra == 0 and ra == 0: # some issue in retrieving these coordinates
            ra = ra_scx
            dec = dec_scx
        
        fits_info[filename] = {
            'EXPOSURE': exposure,
            'RA': ra,
            'DEC': dec,
            'PA': pa
        }

fits_info

{'S0555020101_U.fits': {'EXPOSURE': 4000.0,
  'RA': 187.88196,
  'DEC': 70.709445,
  'PA': 318.45313},
 'S0555220101_M.fits': {'EXPOSURE': 20000.0,
  'RA': 329.72211,
  'DEC': -20.085922,
  'PA': 247.26956},
 'S0303550301_L.fits': {'EXPOSURE': 3535.0,
  'RA': 127.30863,
  'DEC': 50.086445,
  'PA': 296.63327},
 'S0692110301_M.fits': {'EXPOSURE': 3200.0,
  'RA': 257.01655,
  'DEC': 30.00453,
  'PA': 301.17536},
 'S0652550201_B.fits': {'EXPOSURE': 3300.0,
  'RA': 247.47909,
  'DEC': 24.417278,
  'PA': 284.43811},
 'S0674330501_M.fits': {'EXPOSURE': 20700.0,
  'RA': 224.35065,
  'DEC': -35.656309,
  'PA': 112.62868},
 'S0111110501_L.fits': {'EXPOSURE': 6000.0,
  'RA': 35.662443,
  'DEC': -5.1725113,
  'PA': 64.812838},
 'S0093030201_L.fits': {'EXPOSURE': 11700.0,
  'RA': 322.41349,
  'DEC': 0.1015534,
  'PA': 255.74332},
 'S0105260401_L.fits': {'EXPOSURE': 4400.0,
  'RA': 303.05837,
  'DEC': -56.613571,
  'PA': 241.69485},
 'S0830790401_U.fits': {'EXPOSURE': 2200.0,
  'RA': 58.413417,
  'D

In [17]:
obs_id = 'S0021140501_B'+'.fits'
print(str(fits_info[obs_id]['RA']) + ' ' + str(fits_info[obs_id]['DEC']))

263.24477 43.781905


In [32]:
import json 
with open("obs_info_1024_1000", 'w') as json_file:
    json.dump(fits_info, json_file, indent=4)

# Access to XMM-OM catalogue

The XMM-OM catalogue (available here: https://nxsa.esac.esa.int/nxsa-web/#search), contains useful informations about our dataset and can be infered using center coordinates or observation ID.

In [14]:
#pip install --pre --upgrade astroquery 
from astroquery.esa.xmm_newton import XMMNewton 

In [15]:
# get all tables of XMM
tbls = XMMNewton.get_tables(only_names=True) 
om_tbls = [tb for tb in tbls if "om" in tb]
om_tbls

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]


['xsa.v_om_source', 'xsa.v_om_source_cat']

In [16]:
# get all columns of a  table
XMMNewton.get_columns(om_tbls[0])#, XMMNewton.get_columns(om_tbls[1])

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]


['b_ab_mag',
 'b_flux',
 'bii',
 'b_signif',
 'dec',
 'eflag_max',
 'lii',
 'observation_id',
 'observation_oid',
 'om_source_equatorial_spoint',
 'om_source_fov_scircle',
 'om_source_galactic_spoint',
 'om_source_oid',
 'poserr',
 'qflag_comb',
 'ra',
 'src_num',
 'u_ab_mag',
 'u_flux',
 'u_signif',
 'uvm2_ab_mag',
 'uvm2_flux',
 'uvm2_signif',
 'uvw1_ab_mag',
 'uvw1_flux',
 'uvw1_signif',
 'uvw2_ab_mag',
 'uvw2_flux',
 'uvw2_signif',
 'v_ab_mag',
 'v_flux',
 'v_signif']

In [18]:
om_tbl = om_tbls[0]

In [24]:
queryy = f"select * from {om_tbl} where observation_id='0555220101'"
result = XMMNewton.query_xsa_tap(queryy, output_format='csv') # output_file='results10.csv'
result

b_ab_mag,b_flux,bii,b_signif,dec,eflag_max,lii,observation_id,observation_oid,om_source_equatorial_spoint,om_source_fov_scircle,om_source_galactic_spoint,om_source_oid,poserr,qflag_comb,ra,src_num,u_ab_mag,u_flux,u_signif,uvm2_ab_mag,uvm2_flux,uvm2_signif,uvw1_ab_mag,uvw1_flux,uvw1_signif,uvw2_ab_mag,uvw2_flux,uvw2_signif,v_ab_mag,v_flux,v_signif
int64,int64,float64,int64,float64,int64,float64,int64,int64,str71,str80,str71,int64,float64,int64,float64,int64,float64,float64,float64,float64,float64,float64,int64,int64,int64,float64,float64,float64,int64,int64,int64
--,--,-50.05152371647991,--,-20.179052747836682,0,33.394354094997624,555220101,61002,STC-S as Position ICRS GEOCENTER 329.69727654962526 -20.17905274783667,Circle ICRS GEOCENTER 329.69727654962526 -20.17905274783667 0.24999999999999983,STC-S as Position ICRS GEOCENTER 33.39435409499763 -50.0515237164799,28536573,0.10000000149011612,0,329.6972765496255,230,20.03087615966797,8.934109789666785e-17,24.42629623413086,--,--,--,--,--,--,--,--,--,--,--,--
--,--,-50.098777595048006,--,-20.142091213629683,0,33.47981188365792,555220101,61002,STC-S as Position ICRS GEOCENTER 329.7635870958822 -20.142091213629712,Circle ICRS GEOCENTER 329.7635870958822 -20.142091213629712 0.24999999999999983,STC-S as Position ICRS GEOCENTER 33.479811883657895 -50.098777595048006,28536574,0.16088534891605377,128,329.7635870958821,356,21.442337036132812,2.4348187061951478e-17,10.36572551727295,--,--,--,--,--,--,--,--,--,--,--,--
--,--,-50.0518610775814,--,-20.009182955247567,0,33.67370944800089,555220101,61002,STC-S as Position ICRS GEOCENTER 329.75862151722407 -20.009182955247553,Circle ICRS GEOCENTER 329.75862151722407 -20.009182955247553 0.24999999999999983,STC-S as Position ICRS GEOCENTER 33.6737094480009 -50.05186107758138,28536575,0.2101261466741562,0,329.7586215172241,347,22.173215866088867,1.2419824075546867e-17,5.587477207183838,--,--,--,--,--,--,--,--,--,--,--,--
--,--,-49.928912350574365,--,-20.00168762103306,0,33.621192601371554,555220101,61002,STC-S as Position ICRS GEOCENTER 329.623168240916 -20.001687621033078,Circle ICRS GEOCENTER 329.623168240916 -20.001687621033078 0.24999999999999983,STC-S as Position ICRS GEOCENTER 33.621192601371554 -49.928912350574365,28536576,0.4648574888706207,0,329.6231682409161,553,--,--,--,21.839248657226562,3.7453543687596567e-17,3.8187167644500732,--,--,--,--,--,--,--,--,--
--,--,-50.01488772224729,--,-20.190006179087785,0,33.35726050029795,555220101,61002,STC-S as Position ICRS GEOCENTER 329.6522006295579 -20.1900061790878,Circle ICRS GEOCENTER 329.6522006295579 -20.1900061790878 0.24999999999999983,STC-S as Position ICRS GEOCENTER 33.35726050029794 -50.014887722247266,28536577,0.0707106813788414,2,329.6522006295579,122,19.583255767822266,1.3492727953481632e-16,30.543670654296875,--,--,--,--,--,--,--,--,--,--,--,--
--,--,-49.92443083241569,--,-20.100180781080496,0,33.45734365367123,555220101,61002,STC-S as Position ICRS GEOCENTER 329.58269762088133 -20.100180781080486,Circle ICRS GEOCENTER 329.58269762088133 -20.100180781080486 0.24999999999999983,STC-S as Position ICRS GEOCENTER 33.4573436536712 -49.92443083241567,28536578,0.18690325319766998,128,329.5826976208811,29,21.770179748535156,1.800238165724477e-17,7.824778079986572,--,--,--,--,--,--,--,--,--,--,--,--
--,--,-50.03320199080223,--,-20.198050888535036,0,33.353600598531784,555220101,61002,STC-S as Position ICRS GEOCENTER 329.6699091793938 -20.19805088853506,Circle ICRS GEOCENTER 329.6699091793938 -20.19805088853506 0.24999999999999983,STC-S as Position ICRS GEOCENTER 33.353600598531784 -50.033201990802254,28536579,0.1393599957227707,0,329.6699091793938,167,22.190616607666016,1.2222366964343721e-17,5.700700283050537,--,--,--,--,--,--,--,--,--,--,--,--
--,--,-50.05375292848635,--,-19.926906175157526,0,33.81004897737555,555220101,61002,STC-S as Position ICRS GEOCENTER 329.79049683735036 -19.92690617515754,Circle ICRS GEOCENTER 329.79049683735036 -19.92690617515754 0.24999999999999983,STC-S as Position

In [27]:
result['dec', 'ra']

dec,ra
float64,float64
-20.179052747836682,329.6972765496255
-20.142091213629683,329.7635870958821
-20.009182955247567,329.7586215172241
-20.00168762103306,329.6231682409161
-20.190006179087785,329.6522006295579
-20.100180781080496,329.5826976208811
-20.198050888535036,329.6699091793938
-19.926906175157526,329.79049683735053
-19.93182689551846,329.7604302773045


In [28]:
ra_coords = result['ra']
dec_coords = result['dec']

In [29]:
ra_coords.min(), ra_coords.max(), ra_coords.std()

(329.53969657518707, 329.9067213555986, 0.08368771631345341)

In [30]:
dec_coords.min(), dec_coords.max(), dec_coords.std()

(-20.257124697868377, -19.91513369283744, 0.07916997181423098)